In [1]:
# to use inference Engine, specify location of plugins:
# export LD_LIBRARY_PATH=/opt/intel/deeplearning_deploymenttoolkit/deployment_tools/external/mklml_lnx/lib:$LD_LIBRARY_PATH

In [7]:
import pyrealsense2 as rs
import cv2
import numpy as np
import copy
import math
#from appscript import app
import uuid
import os
import test64x64 as ML
import calculator
from pynput.mouse import Button, Controller

# Configure depth and color streams
pipe = rs.pipeline()
#pipeline = rs.pipeline()
profile = pipe.start()
#config = rs.config()
#config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
#config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
#pipeline.start(config)

# Environment:
# OS    : Ubuntu 18.04
# python: 2.7.13
# opencv: 3.4.2

# Values to be initialised
Inited = False
frameW = 300 # frame width
frameH = 300 # frame height
marginX = 10 # margin between region
marginY = 10
gapX = 2 # margin between buttons
gapY = 2

# parameters
cap_region_x_end=0.5 # start point/total width
cap_region_y_end=0.8  # end point/total heighty
capStartX = 0 # keyboard region start x
capStartY = 0 # keyboard region start y
capEndX = None # keyboard region end x
capEndY = None # keyboard region end y


# buttons for capture 1
btnsX = None # btns start x 
btnsY = None # btns start y
btnsW = 600 # btns width
btnsH = 120 # btns height
bW = btnsW / 10  # btn width
bH = btnsH / 3  # btn height
bColor = (0, 255, 255) 
bThick = 2
bTitles = [
            ['lc', 'lo', 'lq', 'lw', 'le', 'l1', 'l2', 'l3', 'l4', 'l5' ],
            ['mc', 'mo', 'mq', 'mw', 'me', 'm1', 'm2', 'm3', 'm4', 'm5' ],
            ['rc', 'ro', 'rq', 'rw', 're', 'r1', 'r2', 'r3', 'r4', 'r5' ]
          ]
btns = [[(bW * j, bH * i, bTitles[i][j] ) for j in range(10)] for i in range(3)] 

# buttons for capture 2
btns2X = None # btns2 start x
btns2Y = None # btns2 start y
btns2W = 300 # btns2 width
btns2H = 120 # btns2 height
bW2 = btns2W / 5  # btn width
bH2 = btns2H / 2  # btn height
bTitles2 = [
            ['vc', 'vo', 'vq', 'vw', 've'],
            ['v1', 'v2', 'v3', 'v4', 'v5']
           ]
btns2 = [[(bW2 * j, bH2 * i, bTitles2[i][j] ) for j in range(5)] for i in range(2)] 

# Mouse Mode
MouseMode = False
mouse_region_x_begin = 0.7
mouse_region_y_end = 0.8
capStartX2 = None # mouse region start x
capStartY2 = 0 # mouse region start y
capEndX2 = None # mouse region end x
capEndY2 = None # mouse region end y

# GUI Mode
GUIMode = False
GUIInited = False

# Demo Mode
DemoMode = False

# variables for getSerialNumber
uid = str(uuid.uuid4())[:8]
inited = False
currentNum = 0
def getSerialNumber(serialPrefix = 'a', startNum = 0, endNum = 90000):
    global uid, inited, currentNum
    
    if not inited:
        currentNum = startNum
        inited = True
    else:
        currentNum += 1
        
    if currentNum > endNum:
        print ("!!! Exceeding Maximum Number !!!")
    
    return serialPrefix + '_' + uid + '_' + str(currentNum)
    
    
#def printThreshold(thr):
 #   print("! Changed threshold to "+str(thr))


# GUI mode to capture a hand
ix, iy = -1, -1
currentClass = None
detectedClass = None
currentClass2 = None
detectedClass2 = None
imgClass = None
redColor = (0,0,255)

def handleClick(event,x,y,flags,param):
    global ix, iy, GUIMode
    
    if GUIMode and event == cv2.EVENT_LBUTTONDOWN:
        ix,iy = x,y
        detectBtn(ix, iy)

def detectBtn(ix, iy):
    global btnsX, btnsY, btnsW, btnsH, btns, btns2X, btns2Y, btns2W, btns2H, btns2
    global currentClass, detectedClass, currentClass2, detectedClass2
    
    if ((btnsX < ix) and (ix < btnsX + btnsW) and (btnsY < iy) and (iy < btnsY + btnsH)):
        nX = (ix - btnsX) // bW
        nY = (iy - btnsY) // bH
        currentClass = detectedClass = btns[nY][nX][2]
    
    elif MouseMode and ( btns2X < ix) and (ix < btns2X + btns2W) and (btns2Y < iy) and (iy < btns2Y + btns2H):
        nX = (ix - btns2X) // bW2
        nY = (iy - btns2Y) // bH2
        currentClass2 = detectedClass2 = btns2[nY][nX][2] 
    
def drawBtns(frame, btns, btnsX, btnsY, bW, bH, Class):
    numRows = len(btns)
    numCols = len(btns[0])
    
    for i in range(numRows):
        for j in range(numCols):
            drawBtn(frame, btns[i][j], btnsX, btnsY, bW, bH, Class)
            
def drawBtn(frame, btn, btnsX, btnsY, bW, bH, Class):
    global bColor, bThick, currentClass2, btns2X, btns2Y, bW2, bH2, redColor, gapX, gapY

    startPoint = (btnsX + btn[0] + gapX, btnsY + btn[1] + gapY)
    endPoint = (btnsX + btn[0] + bW - gapX, btnsY + btn[1] + bH - gapY)
    bTitle = btn[2]

    if Class == bTitle:
        color = redColor
    else:
        color = bColor

    cv2.rectangle(frame, startPoint, endPoint, color, bThick)
    cv2.putText(frame, bTitle, (startPoint[0]+8, startPoint[1]+32), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

# file path for cropped image
outDirPath = os.path.join('fdata', 'pic')
csvFilePath = os.path.join('fdata', 'fdata.csv')
csvFile = None

# margin for cropped image
margin = 10             
def getCrop(x, y, w, h, margin, img):
    # make a crop a square to prevent a shape destortion 
    if w > h: 
        delta = int((w-h)/2)

        y1 = y - delta - margin
        if y1 < 0: y1 = y

        y2 = y + h + delta + margin
        if y2 > img.shape[0]: y2 = y + h

        x1 = x - margin
        if x1 < 0: x1 = x

        x2 = x + w + margin
        if x2 > img.shape[1]: x2 = x + w

        crop = img[y1 : y2, x1 : x2]
    else:
        delta = int((h-w)/2)

        y1 = y - margin
        if y1 < 0: y1 = y

        y2 = y + h + margin
        if y2 > img.shape[0]: y2 = y + h

        x1 = x - delta - margin
        if x1 < 0: x1 = x

        x2 = x + w + delta + margin
        if x2 > img.shape[1]: x2 = x + w

        crop = img[y1 : y2, x1 : x2]
        
    return crop

def saveCrop(crop, imgClass):
    global outDirPath
    
    # make a crop a 200x200 pixel
    f_im = cv2.resize(crop,(200, 200), interpolation=cv2.INTER_CUBIC)

    # save into image file
    fileName = getSerialNumber('a', 0, 90000) + '_' + imgClass + '.jpg'
    out_path = os.path.join(outDirPath, fileName)
    cv2.imwrite(out_path, f_im)
    print out_path
    
    return fileName

def saveCsv(fileName, imgClass):
    global csvFilePath
    
    # save into csv file
    csvFile = open(csvFilePath, 'a') # to append a data to an existing csv file
    csvFile.write(fileName + ', ' + imgClass + '\r\n')
    csvFile.close()
    
def initParams(frame):
    global cap_region_x_end, cap_region_y_end, capStartX, capStartY, capEndX, capEndY
    global MouseMode, mouse_region_x_begin, mouse_region_y_end, capStartX2, capStartY2, capEndX2, capEndY2
    global frameW, frameH, btnsX, btnsY, btns2X, btns2Y, marginX, marginY
    
    frameW = frame.shape[1]
    frameH = frame.shape[0]
    capStartX = 0
    capStartY = 0
    capEndX = int(cap_region_x_end * frameW)
    capEndY = int(cap_region_y_end * frameH)
    capStartX2 = int(mouse_region_x_begin * frameW)
    capStartY2 = 0
    capEndX2 = frameW
    capEndY2 = int(mouse_region_y_end * frameH)
    
    btnsX = marginX
    btnsY = capEndY + marginY
    btns2X = capStartX2 + marginX
    btns2Y = frameH - btns2H - marginY
    
# track hand by skin color
skin_ycrcb_mint = np.array((0, 133, 77))
skin_ycrcb_maxt = np.array((255, 173, 127))

def trackHand(img):
    global skin_ycrcb_mint, skin_ycrcb_maxt
    
    im_ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)
    skin_ycrcb = cv2.inRange(im_ycrcb, skin_ycrcb_mint, skin_ycrcb_maxt)    
    _, contours, _ = cv2.findContours(skin_ycrcb, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    ## get the coutours
    length = len(contours)
    maxArea = -1
    res = None
    if length > 0:
        for i in range(length):  # find the biggest contour (according to area)
            temp = contours[i]
            area = cv2.contourArea(temp)
            if area > maxArea:
                maxArea = area
                ci = i

        # draw the bounding rect
        res = contours[ci]
    return res

def drawGuide(frame, btnsX, btnsY, btnsW, marginX):
    cv2.putText(frame,"d: Demo mode", 
                (btnsX + btnsW + marginX, btnsY + 25), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                1,(0,255,255),2,cv2.LINE_AA)
    cv2.putText(frame,"g: GUI mode", 
                (btnsX + btnsW + marginX, btnsY + 55), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                1,(0,255,255),2,cv2.LINE_AA)
    cv2.putText(frame,"m: Mouse mode.", 
                (btnsX + btnsW + marginX, btnsY + 85), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                1,(0,255,255),2,cv2.LINE_AA)
    cv2.putText(frame,"esc: quit", 
                (btnsX + btnsW + marginX, btnsY + 115),  
                cv2.FONT_HERSHEY_SIMPLEX, 
                1,(0,255,255),2,cv2.LINE_AA)
    
def drawText(frame, startPoint, text):
    cv2.putText(frame, text, 
                startPoint, 
                cv2.FONT_HERSHEY_SIMPLEX, 
                2,(0,255,255),2,cv2.LINE_AA)    
# Camera
#camera = cv2.VideoCapture(0)
#camera.set(10,200)

cv2.namedWindow('main')
cv2.setMouseCallback('main', handleClick)

# main loop
prevGes = None
finalGes = None
framerate = None
framecount = 0

# Cal mode to calculate
CalMode = False
gesCount = 0
ges_to_cal = {'lc':'', 'lo':'', 'lq':'', 'lw':'del', 'le':'', 'l1':'1', 'l2':'2', 'l3':'3', 'l4':'4', 'l5':'5',
            'mc':'', 'mo':'', 'mq':'', 'mw':'del', 'me':'', 'm1':'6', 'm2':'7', 'm3':'8', 'm4':'9', 'm5':'0' ,
            'rc':'', 'ro':'', 'rq':'=', 'rw':'del', 're':'ce', 'r1':'+', 'r2':'-', 'r3':'*', 'r4':'/', 'r5':'' }

calResult = ''
calStr = ''
calFinished = False

#while camera.isOpened():
try:
    for i in range(0, 100):
        frames = pipe.wait_for_frames()
        frame = np.asanyarray(frames.get_data())
    #for f in frames:
       # print(f.profile)
    
    
    
    #while True:

        # Wait for a coherent pair of frames: depth and color
        #ret, frames = pipeline.wait_for_frames()
        #frame = np.asanyarray(frames.get_data())
   # threshold = cv2.getTrackbarPos('trh1', 'trackbar')
        frame = cv2.bilateralFilter(frame, 5, 50, 100)  # smoothing filter
        frame = cv2.flip(frame, 1)  # flip the frame horizontally
    
    # init
        if not Inited:
            initParams(frame)
            Inited = True

        # keyboard region
        cv2.rectangle(frame, (capStartX, capStartY), (capEndX, capEndY), (255, 0, 0), 2)

        # mouse region
        if MouseMode:
            cv2.rectangle(frame, (capStartX2, capStartY2), (capEndX2, capEndY2), (255, 0, 0), 2)

        if GUIMode:
            if not GUIInited:
                if not os.path.exists(outDirPath):
                    os.makedirs(outDirPath)
                GUIInited = True

            drawBtns(frame, btns, btnsX, btnsY, bW, bH, currentClass)

            if MouseMode:
                drawBtns(frame, btns2, btns2X, btns2Y, bW2, bH2, currentClass2)

        #  Main operation
        if True:
            img = frame[capStartY : capEndY, capStartX : capEndX]  # clip the ROI
            img = copy.deepcopy(img)
            res = trackHand(img)
            (x,y,w,h) = cv2.boundingRect(res)
            cv2.rectangle(frame, 
                          (capStartX + x - margin, y - margin), 
                          (capStartX + x + w + margin, y + h + margin), 
                          (0, 255, 255), 2)

            #Find moments of the largest contour
            moments = cv2.moments(res)

            # Central mass of first order moments
            if moments['m00']!=0:
                cx = int(moments['m10']/moments['m00']) # cx = M10/M00
                cy = int(moments['m01']/moments['m00']) # cy = M01/M00

                #Draw center mass at capture
                centerMass=(cx,cy)   
                cv2.circle(img,centerMass,1,[100,0,255],1)

            # mouse
            if MouseMode:
                img2 = frame[capStartY2 : capEndY2, capStartX2: capEndX2]  # clip the ROI
                img2 = copy.deepcopy(img2)
                res2 = trackHand(img2)
                (x2,y2,w2,h2) = cv2.boundingRect(res2)
                cv2.rectangle(frame, 
                              (capStartX2 + x2 - margin, y2 - margin), 
                              (capStartX2 + x2 + w2 + margin, y2 + h2 + margin), 
                              (255, 0, 255), 2)

                #Find moments of the largest contour
                moments2 = cv2.moments(res2)

                #Central mass of first order moments
                if moments2['m00']!=0:
                    cx2 = int(moments2['m10']/moments2['m00']) # cx = M10/M00
                    cy2 = int(moments2['m01']/moments2['m00']) # cy = M01/M00

                    #Draw center mass at capture
                    centerMass3=(cx2,cy2)
                    cv2.circle(img2,centerMass3,1,[100,0,255],1)

                    #Draw center mass at frame
                    centerMass2=(capStartX2 + cx2,cy2) 
                    cv2.circle(frame,centerMass2,5,[100,0,255],2)

                    # Set pointer position
                    if DemoMode: mouse.position = (cx2*5, cy2*5)


            drawGuide(frame, btnsX, btnsY, btnsW, marginX)



        # Keyboard OP
        k = cv2.waitKey(10)
        if k == 27:  # press ESC to exit
            if csvFile and (not csvFile.closed):
                csvFile.close()
            break

        elif k == ord('g'): # GUI mode to capture a hand
            if not GUIMode: 
                GUIMode = True
                print '!!! GUI Mode on !!!'
            else:
                GUIMode = False
                print '!!! GUI Mode off !!!'

        elif k == ord('m'): # Mouse mode to capture a hand
            if not MouseMode: 
                MouseMode = True
                mouse = Controller()
                print '!!! Mouse Mode on !!!'
            else:
                MouseMode = False
                print '!!! Mouse Mode off !!!'

        elif k == ord('d'): # Demo mode
            if not DemoMode: 
                DemoMode = True
                print '!!! Demo Mode on !!!'
            else:
                DemoMode = False
                print '!!! Demo Mode off !!!'

        elif k == ord('c'): # Cal mode to calculate
            if not CalMode: 
                CalMode = True
                print '!!! Calculator Mode on !!!'
            else:
                CalMode = False
                print '!!! Calculator Mode off !!!'

        elif GUIMode:
            if detectedClass:
                imgClass = detectedClass
                X = x
                Y = y
                W = w
                H = h
                Img = img

            elif detectedClass2:
                imgClass = detectedClass2 
                X = x2
                Y = y2
                W = w2
                H = h2
                Img = img2

            if imgClass:
                # get a crop
                crop = getCrop(X, Y, W, H, margin, Img)

                # make a crop a 200x200 pixel
                fileName = saveCrop(crop, imgClass)

                # save into csv file
                saveCsv(fileName, imgClass)

                # restore the initial state
                imgClass = detectedClass = detectedClass2 = None

        elif DemoMode:
            X = x
            Y = y
            W = w
            H = h
            Img = img

            if framecount % 3 == 0:
                framecount = 0

                # get crop and predict by ML
                #crop = getCrop(X, Y, W, H, margin, Img) ## problem
                crop = Img[y:y+h, x:x+w]
                predictedGes = str(ML.predict(crop)).strip()

                if prevGes != predictedGes:
                    prevGes = predictedGes
                    gesCount = 0

                if prevGes == predictedGes and finalGes != predictedGes:
                    gesCount += 1

                if gesCount >= 3:
                    gesCount = 0
                    finalGes = predictedGes
                    finalCal = ges_to_cal[finalGes]

                    if CalMode:
                        if finalCal:
                            if finalCal == '=':
                                calResult = calculator.calculate(calStr)
                                calStr = calStr + ' = ' + calResult 
                                calFinished = True
                            elif finalCal == 'del':
                                calStr = calStr[:-1]
                            elif finalCal == 'ce':
                                calResult = ''
                                calStr = ''
                                calFinished = False
                            elif not calFinished:
                                calStr = calStr + finalCal

            framecount += 1

            if finalGes:
                drawText(frame, (100, 50), finalGes + ': ' + finalCal)

            if CalMode and calStr:
                drawText(frame, (100, 100), calStr)  

# Show images
            cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
            cv2.imshow('main', frame)
            cv2.waitKey(1)

finally:

    # Stop streaming
    pipe.stop()

AttributeError: 'NoneType' object has no attribute 'shape'